In [53]:
import random
import pandas as pd
import numpy as np
from  flask_restful import Api, Resource, reqparse
from collections import defaultdict
app = Flask(__name__)
api = Api(app)

parser = reqparse.RequestParser()
parser.add_argument('word')
parser.add_argument('wordlist', action='append')


In [75]:
#import CMU dictionary idea for method from:
# https://github.com/aparrish/gen-text-workshop/blob/master/cmu_pronouncing_dictionary_notes.md
rhymedict = defaultdict(list)
worddict = {}
with open('cmudict-0.7b.txt') as f:
    for line in f:
        #skip comments
        if(line[:3]==';;;'):
            continue
        #split elements on the double space
        word, sounds = line[:-1].split('  ')
        sounds = sounds.split(' ')
        if(len(sounds)==1):
            rhyme = sounds[0]
        else:
            rhyme = sounds[-2] + sounds[-1]
        rhymedict[rhyme].append(word)
        worddict[word] = sounds

In [ ]:
class pick_random(Resource):
    '''Returns a random word from a list of inputs.
    
    Does not check for incorrect user inputs.'''
    #We don't check for misinputs due to that api interface feeding strings.
    #Number check could be done by attempting to convert all list elements to int
    
    def post(self):
        args = parser.parse_args()
        wordlist = args['wordlist']
        return wordlist[random.randrange(len(wordlist))]

In [ ]:
class rhymes(Resource):
    '''returns a list of words that rhyme with the input
    
    we'll use a simple definition of rhyming words as words that share the last phonetic sounds'''
    
    def post(self):
        args = parser.parse_args()
        word = args['word']
        rhyme = worddict[word]
        if(len(rhyme)==1):
            rhymelist = rhymedict[rhyme]
        else:
            rhymelist = rhymedict[rhyme[-2]+rhyme[-1]]
        return rhymelist

In [ ]:
#finish building api
api.add_resource(pick_random, '/pickrnd')
api.add_resource(rhymes, '/rhyme')

if __name__ == '__main__':
    app.run()


In [69]:
phrase = 'blah blah blah'
phrase.split(' ')

['blah', 'blah', 'blah']

In [54]:
test = 'word(1)'
'(' in test

True

In [21]:
random.randrange(5)

0

In [28]:
type(test)

list

In [29]:
int in test

False

In [33]:
testnp = np.array(test)
testnp

array(['h', 'e', 'l', 'l', 'o', '1'],
      dtype='<U1')

In [37]:
testpd = pd.Series(test)
testpd

0    h
1    e
2    l
3    l
4    o
5    1
dtype: object

In [45]:
testpd.

object:dense    1
dtype: int64